#### Необходимые библиотеки для занятия    
Обычно <b>ставятся автоматически</b> при установке jupyter notebook:
<ul>
<li><i>numpy</i> - для работы с массивами

```!pip3 install numpy```
</li>
<li><i>sklearn</i> - блиблиотека для работы с данными, содержит большинство алгоритмов машинного обучения</li>

```!pip3 install sklearn```

</li>
<li><i>pandas</i> - библиотека для удобной работы с данными как с DataFrame

```!pip3 install pandas```
</li>
<li><i>matplotlib</i> - библиотека для визуализации

```!pip3 install matplotlib```
</li>
</ul>

Обычно <b>не ставятся автоматически</b> при установке jupyter notebook:
<ul>
<li><i>rusenttokenize</i> - для разбиения текста на предложения (русский язык)

```!pip3 install rusenttokenize```
</li>
<li><i>nltk</i> - фреймворк для обработки ЕЯ

```!pip3 install nltk```

Также необходимо выполнить следующие команды, чтобы загрузить данные:


```import nltk```

```nltk.download('treebank')```

```nltk.download('stopwords')```

```nltk.download('punkt')```

<li><i>python-crfsuite</i> - для использования CRF</li>

```!pip3 install python-crfsuite```
</li>
<li><i>pymorphy2</i> - морфологический анализатор русского языка

```!pip3 install pymorphy2```
</li>
</ul>

### For colab:
Install libs

In [0]:
!pip3 install nltk
import nltk
nltk.download('treebank')
nltk.download('stopwords')
nltk.download('punkt')

In [0]:
!pip3 install python-crfsuite
!pip3 install pymorphy2

In [0]:
!pip3 install rusenttokenize

### Load data

In [0]:
!wget -O negative.txt https://raw.githubusercontent.com/king-menin/nlp-hse-winter2018/master/lecture%201.%20intro%20to%20nlp/negative.txt
!wget -O positive.txt https://raw.githubusercontent.com/king-menin/nlp-hse-winter2018/master/lecture%201.%20intro%20to%20nlp/positive.txt

In [0]:
!ls

negative.txt  positive.txt  sample_data


# Часть 1. Предобработка текста и классификация

In [0]:
import warnings
warnings.filterwarnings('ignore')


import numpy as np
np.random.seed(1)

## 1.1 Прочитайте текст в файлах positive.txt и negative.txt

In [0]:
with open("positive.txt", "r", encoding="utf-8") as file:
    positive_plain = <your code here>

with open("negative.txt", "r", encoding="utf-8") as file:
    negative_plain = <your code here>

In [0]:
print(positive_plain[:400], negative_plain[:400], sep="\n\n")

## 1.2. Разбейте данные на предложения

Для токенизации используйте nltk.sent_tokenize. чтобы удалить имен пользователей из сообщений напишите регулярное выражение.

<b>Классы символов в регулярных выражениях</b>:

[A-Z] – символы верхнего регистра (латиница)

[a-z] – символы нижнего регистра (латиница)

[А-Я] – символы верхнего регистра (кириллица)

[а-я] – символы нижнего регистра (кириллица)

[0-9] или \d – цифра

[^0-9] или \D – любой символ, кроме цифры

. – Один любой символ, кроме новой строки \n.

? – 0 или 1 вхождение шаблона слева

\+ – 1 и более вхождений шаблона слева

\* – 0 и более вхождений шаблона слева

\w – Любая цифра или буква (\W — все, кроме буквы или цифры)

\d – Любая цифра [0-9] (\D — все, кроме цифры)

\s – Любой пробельный символ (\S — любой непробельнй символ)

\b – Граница слова

[..] – дин из символов в скобках ([^..] — любой символ, кроме тех, что в скобках)

\ – Экранирование специальных символов (\. означает точку или \+ — знак «плюс»)

^ и $ – Начало и конец строки соответственно

{n,m} – От n до m вхождений ({,m} — от 0 до m)

a|b – Соответствует a или b

() – Группирует выражение и возвращает найденный текст

\t, \n, \r – Символ табуляции, новой строки и возврата каретки соответственно

In [0]:
import re
# from nltk import sent_tokenize
from rusenttokenize import ru_sent_tokenize


def split_data(text):
    # Избавляемся от имен пользователей, указанных в письмах
    name = <your code here>
    res = name.sub("", text)
    # Удалите лишние переносы строк и разбейте на предложения по знаку "."
    # res = sent_tokenize(res.replace("\n", ""))
    res = ru_sent_tokenize(res.replace("\n", ""))
    return res

In [0]:
positive = split_data(positive_plain)
negative = split_data(negative_plain)

In [0]:
len(positive), len(negative)

In [0]:
pos_labels = [1] * len(positive)

In [0]:
neg_labels = [0] * len(negative)

Объединим все в один список

In [0]:
all_text = positive + negative
all_labels = pos_labels + neg_labels

In [0]:
len(all_text)

## 1.3. Удалите пустые строки, если они есть

In [0]:
all_text_ = []
all_labels_ = []

<your code here>

all_text = all_text_
all_labels = all_labels_

## 1. 4. Предварительный анализ коллекции

#### Средняя длина предложений

In [0]:
import pandas as pd

In [0]:
df = pd.DataFrame(all_text)

In [0]:
len_data = df[0].apply(len)

In [0]:
len_data.describe()

#### Длины текстов в символах

In [0]:
import matplotlib.pyplot as plt

%pylab inline

In [0]:
fig, ax = plt.subplots()

n, bins, patches = ax.hist(len_data.tolist())

fig.show()

#### Самые частые слова

Токенизируйте предложения на слова (nltk.word_tokenize)

In [0]:
from nltk import word_tokenize


all_text = [word_tokenize(line) for line in all_text]

In [0]:
from nltk import FreqDist


n_types = []
n_tokens = []
fd = FreqDist()
for line in all_text:
    fd.update(line)
    n_types.append(len(fd))
    n_tokens.append(sum(list(fd.values())))
for i in fd.most_common(10):
    print(i)

#### Закон Ципфа

В любом достаточно большом тексте ранг типа обратно пропорционален его частоте: f=a/r

f – частота типа, r – ранг типа, a – параметр, для славянских языков – около 0.07

In [0]:
freqs = list(fd.values())
freqs = sorted(freqs, reverse = True)

fig, ax = plt.subplots()
ax.plot(freqs[:300], range(300))
plt.show()

#### Закон Хипса

С увеличением длины текста (количества токенов), количество типов увеличивается в соответствии с законом: |V|=K∗N^b

N – число токенов, |V| – количество типов в словаре, K,b – параметры, обычно K∈[10,100],b∈[0.4,0.6]

In [0]:
fig, ax = plt.subplots()
ax.plot(n_types, n_tokens)
plt.show()

## 1.5. Подготовим данные для nltk.NaiveBayesClassifier

Классификатор принимает данные о предложении в виде словаря {"слово": #количество встреч в предожении}

Используйсте Counter. Пример работы:

`Counter('abracadabra')`

>Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [0]:
from collections import Counter

In [0]:
all_text = <your code here>

## 1.6. Обучим модель

In [0]:
from sklearn.metrics import accuracy_score
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

In [0]:
train_indecies, test_indecies = train_test_split(np.arange(len(all_text)), test_size=0.3, random_state=1)

In [0]:
all_text = np.array(all_text)
all_labels = np.array(all_labels)

X_train, X_test = all_text[train_indecies], all_text[test_indecies]
y_train, y_test = all_labels[train_indecies], all_labels[test_indecies]

In [0]:
clf = NaiveBayesClassifier.train(zip(X_train, y_train))

### Протестируем качество классификации

In [0]:
pred = clf.classify_many(X_test)
accuracy_score(y_test, pred)

## 1.7. Добавим признаков

Для каждого примера добавим количество частей речи в нем. Если мы встретили знак пунктуации, то обозначим его как 'PNCT'.

In [0]:
import pymorphy2

In [0]:
morph = pymorphy2.analyzer.MorphAnalyzer()

Используйте pymorphy2.analyzer.MorphAnalyzer.parse

In [0]:
all_text_with_pos = []
for sample in all_text:
    update = Counter(sample)
    for word, count in sample.items():
        <your code here>
    all_text_with_pos.append(update)

In [0]:
all_text_with_pos = np.array(all_text_with_pos)
all_labels = np.array(all_labels)

X_train, X_test = all_text_with_pos[train_indecies], all_text_with_pos[test_indecies]
y_train, y_test = all_labels[train_indecies], all_labels[test_indecies]

In [0]:
clf = NaiveBayesClassifier.train(zip(X_train, y_train))

In [0]:
pred = clf.classify_many(X_test)
accuracy_score(y_test, pred)

### Попробуем лемматизировать слова

Обучите тот же классификатор но на примерах, где все слова в нормальной форме.

In [0]:
all_text_with_pos = []
for sample in all_text:
    update = Counter()
    for word, count in sample.items():
        <your code here>
    all_text_with_pos.append(update)

In [0]:
all_text_with_pos = np.array(all_text_with_pos)
all_labels = np.array(all_labels)

X_train, X_test = all_text_with_pos[train_indecies], all_text_with_pos[test_indecies]
y_train, y_test = all_labels[train_indecies], all_labels[test_indecies]

In [0]:
clf = NaiveBayesClassifier.train(zip(X_train, y_train))

pred = clf.classify_many(X_test)
accuracy_score(y_test, pred)

### Удалим стоп-слова

In [0]:
from nltk.corpus import stopwords

stops = stopwords.words("russian")

In [0]:
all_text_without_stops = []
for sample in all_text:
    update = Counter()
    for word, count in sample.items():
        <your code here>
    all_text_without_stops.append(update)

In [0]:
all_text_without_stops = np.array(all_text_without_stops)
all_labels = np.array(all_labels)

X_train, X_test = all_text_without_stops[train_indecies], all_text_without_stops[test_indecies]
y_train, y_test = all_labels[train_indecies], all_labels[test_indecies]

In [0]:
clf = NaiveBayesClassifier.train(zip(X_train, y_train))

pred = clf.classify_many(X_test)
accuracy_score(y_test, pred)

### Как менялось качество? Почему?

In [0]:
<your answer here>

# Часть 2. Использование знаний морфологии для генерации текста

Задание: написать шаблон вида: {person} {action} {count} {target}. Где

Person - лицо которое выполняет действие action. Каждое такое действие может быть в трех временах (настоящее, прошедшее и будущее). Действие выполняется с целью target. Такая цель - это некоторый объект или объекты числом count. count>0.

Используйте make_agree_with_number, parse и inflect из библиотеки pymorphy2.

Времена глаголов в документации pymorphy2: past (прошедшее), pres (настоящее), futr (будущее).

In [0]:
import pymorphy2
morph = pymorphy2.analyzer.MorphAnalyzer()

Согласование существительного с числом.

In [0]:
butyavka = morph.parse("бутявка")[0]
butyavka.make_agree_with_number(2)

Чтобы поставить слово в нужную форму используется inflect

In [0]:
butyavka.inflect({'gent'})  # нет кого? (родительный падеж)

In [0]:
butyavka.inflect({'plur', 'gent'})

In [0]:
def genarate(person, action, time, count, target):
    # Поставьте action в нужное время
    action = <your code here>
    # Согласуйте target с числом
    target = <your code here>
    return <your code here>

In [0]:
genarate("Антон", "купить", "past", 5, "товар")

# Часть 3. Статистические морфологические анализаторы

## 3.1. Загрузим данные из nltk.treebank

In [0]:
from nltk.corpus import treebank

In [0]:
sentences = treebank.tagged_sents()

## 3.2. Выделим признаки

In [0]:
def features(sentence, index):
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'is_all_lower': sentence[index].islower(),
        'is_first_cap': sentence[index][0].upper()
    }

In [0]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [0]:
# Split the dataset for training and testing
cutoff = int(.75 * len(sentences))
training_sentences = sentences[:cutoff]
test_sentences = sentences[cutoff:]


def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y
 
X, y = transform_to_dataset(training_sentences)

## 3.3. В качестве классификатора используем DecisionTreeClassifier

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline


clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])


# Используем не все примеры (может не хватить оперативной памяти или долго обучаться)
bound = 10000
clf.fit(X[:bound], y[:bound])

In [0]:
X_test, y_test = transform_to_dataset(test_sentences)

In [0]:
clf.score(X_test, y_test)

## 3.4. Используйте RandomForestClassifier

In [0]:
from sklearn.ensemble import RandomForestClassifier


clf = <your code here>

# Используем не все примеры (может не хватить оперативной памяти или долго обучаться)
bound = 10000
clf.fit(X[:bound], y[:bound])

In [0]:
clf.score(X_test, y_test)

## 3.5. Используем с помощью CRF

In [0]:
len(sentences)

In [0]:
train, test = sentences[:-cutoff], sentences[-cutoff:]

In [0]:
from nltk.tag import CRFTagger

In [0]:
ct = CRFTagger()

In [0]:
ct.train(train ,'model.crf.tagger')

In [0]:
ct.evaluate(test)